### <b>Turn Crestlines into Shapefile</b>

In [1]:
import numpy as np
from shapely.geometry import LineString
import geopandas as gpd
from FeatureLines import ReadCrestLine
import matplotlib.pyplot as plt

In [3]:
project_filename = '' # name of project, must be same as that used in previous files!

In [12]:
parent_folder = "" # 'stitch_together_for_231206/'
name_to_filename_dictionary = {
    # this is a dictionary of crest line files we used (note the 'ravines')
    'main mound left' # preferred display name of the crest line
        : {
        'filename': ''} # name of crest line file,    
}

In [3]:
def undo_centering(names_readable : list[str], list_of_lines : list[LineString], 
  movement_to_undo : list[float]):
  for name in names_readable:
    print(f"working on: {name}")
    filename = f"{name_to_filename_dictionary[name]['filename']}.txt"
    print(f"filename: {filename}")
    open(filename)
    crestline_V_3d, crestline_E = ReadCrestLine(f"{name_to_filename_dictionary[name]['filename']}.txt")
    print(f"we got: \n\t{crestline_V_3d.shape[0]} points, \n\t{crestline_E.shape[0]} lines")
    if movement_to_undo != [0.0, 0.0, 0.0]:
        for (start_index, end_index, _) in crestline_E:
            start_point = [
                crestline_V_3d[start_index][0] + movement_to_undo[0],
                crestline_V_3d[start_index][1] + movement_to_undo[1]]
            end_point = [
                crestline_V_3d[end_index][0] + movement_to_undo[0],
                crestline_V_3d[end_index][1] + movement_to_undo[1]]
            line = LineString([start_point, end_point])
            list_of_lines.append(line)  

In [ ]:
# the following 'centering_movement_to_undo' displacement vector comes from point_cloud_to_mesh.ipynb
# note that in this code implementation, the displacement vector is the same for all TGB crest line files we work with
# this because the crest lines in each crest line file are from sections of the same mesh
# and therefore share its displacement from the origin
centering_movement_to_undo = [4.37392254e+05, 1.43946825e+07, 1.95543722e+03]
lines_d_13 : list[LineString] = []
region_name_list_d_13 = ['main mound left', 'main mound right', 'left mound left', 'right mound', 'right deserts',
                         'left mound right', 'lower fields']
undo_centering(region_name_list_d_13, lines_d_13, centering_movement_to_undo)

In [75]:
# SET precision to 6 decimal places
gpd.options.display_precision = 6
# lines_frame.crs = 'EPSG:4326'
lines_frame_d_13 = gpd.GeoDataFrame(data={'geometry': lines_d_13}, crs="EPSG:4326")

In [ ]:
f, ax = plt.subplots(figsize=(128, 64))
lines_frame_d_13.plot(aspect=1, color='green', ax=ax)
ax.set_axis_off()

In [13]:
"""Save crest lines as Shapefile"""
lines_frame_d_13.to_file(f"{project_filename}.shp")